<a href="https://colab.research.google.com/github/FrancescaSrc/AIprojects/blob/master/CharacterLevel_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. I will train on Dante's Divina Commedia. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.


In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
!ls 'drive/My Drive/Colabs/data/'

anna.txt  dante.txt


In [0]:
with open('drive/My Drive/Colabs/data/dante.txt', 'r') as f:
    text = f.read()

In [0]:
text[:100]

'\ufeff\nLA DIVINA COMMEDIA DI DANTE ALIGHIERI\n\nIncipit Comoedia Dantis Alagherii,\nFlorentini natione, non '

Create a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [0]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars= tuple(set(text)) #this is the vocabulary, the set function will get rid of duplicates, selecting unique charaters
print('chars', chars[:100])
int2char= dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
print('char2int', char2int)

# encode the text
encoded = np.array([char2int[ch] for ch in text])
print('encoded', encoded[:100])

chars ('L', '-', 'H', 'z', '`', 'ň', 's', ',', 'f', 'ó', 'h', 'N', 'ů', '#', 'i', 'J', '>', '\ufeff', 'e', 'l', 'B', 'I', ')', ' ', 'p', '?', 'Q', 'P', 'Z', '3', 'U', 'D', 'R', 'V', 'w', 'o', 'b', '@', 'x', 't', 'c', '8', 'y', '0', 'G', 'M', '\n', 'A', ':', '2', 'j', 'O', 'a', '<', '7', '[', 'd', 'W', 'X', 'ŕ', 'q', '1', '(', '/', '!', 'F', 'm', 'Ť', '9', 'ě', 'C', 'č', 'v', 'S', ']', 'E', 'ť', '.', '"', 'k', 'é', 'u', ';', 'r', 'n', 'g', "'", 'ď', 'T')
char2int {'L': 0, '-': 1, 'H': 2, 'z': 3, '`': 4, 'ň': 5, 's': 6, ',': 7, 'f': 8, 'ó': 9, 'h': 10, 'N': 11, 'ů': 12, '#': 13, 'i': 14, 'J': 15, '>': 16, '\ufeff': 17, 'e': 18, 'l': 19, 'B': 20, 'I': 21, ')': 22, ' ': 23, 'p': 24, '?': 25, 'Q': 26, 'P': 27, 'Z': 28, '3': 29, 'U': 30, 'D': 31, 'R': 32, 'V': 33, 'w': 34, 'o': 35, 'b': 36, '@': 37, 'x': 38, 't': 39, 'c': 40, '8': 41, 'y': 42, '0': 43, 'G': 44, 'M': 45, '\n': 46, 'A': 47, ':': 48, '2': 49, 'j': 50, 'O': 51, 'a': 52, '<': 53, '7': 54, '[': 55, 'd': 56, 'W': 57, 'X': 58, 'ŕ': 

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array [* means all the args in an array] 
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    # np.arange retuns an array ex. np.arange(3,7) returns array([3, 4, 5, 6])
    # numpy.arange([start, ]stop, [step, ] dtype=None)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    batch_size_total = batch_size * seq_length
    n_batch = len(arr)//batch_size_total
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:n_batch*batch_size_total]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size,-1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            #make the last element of y equal to the first element in the array
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        
       
       
        yield x, y

In [0]:
# test
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [0]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[17 46  0 47 23 31 21 33 21 11]
 [10 18 23 72 14 84 40 14 46 23]
 [ 8 19 14 39 39 35  7 46 23 23]
 [72 83 52 23 19 86 75 83 66 35]
 [10 86 18  4 23 66 35 39 35 23]
 [ 6 35  8  8 14 52 39 52 23 18]
 [23 40 35 66 86 81 35 66 23  6]
 [23 24 18 83 23 66 14 85 19 14]]

y
 [[46  0 47 23 31 21 33 21 11 47]
 [18 23 72 14 84 40 14 46 23 23]
 [19 14 39 39 35  7 46 23 23  6]
 [83 52 23 19 86 75 83 66 35 23]
 [86 18  4 23 66 35 39 35 23  6]
 [35  8  8 14 52 39 52 23 18 23]
 [40 35 66 86 81 35 66 23  6 86]
 [24 18 83 23 66 14 85 19 14 35]]


In [0]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## put x through the fully-connected layer
        out = self.fc(out)
        
       
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [0]:
#train the RNN
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses))) 

In [0]:
print(train_on_gpu)

True


In [0]:
# define and print the net
n_hidden=1024
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(89, 1024, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=1024, out_features=89, bias=True)
)


In [0]:
batch_size =128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.0848... Val Loss: 3.0480
Epoch: 1/20... Step: 20... Loss: 3.0492... Val Loss: 3.0360
Epoch: 1/20... Step: 30... Loss: 3.0319... Val Loss: 3.0247
Epoch: 2/20... Step: 40... Loss: 3.0526... Val Loss: 3.0196
Epoch: 2/20... Step: 50... Loss: 2.9987... Val Loss: 3.0078
Epoch: 2/20... Step: 60... Loss: 3.0206... Val Loss: 2.9924
Epoch: 2/20... Step: 70... Loss: 2.9553... Val Loss: 2.9466
Epoch: 3/20... Step: 80... Loss: 2.8286... Val Loss: 2.8419
Epoch: 3/20... Step: 90... Loss: 2.6757... Val Loss: 2.6717
Epoch: 3/20... Step: 100... Loss: 2.5327... Val Loss: 2.5616
Epoch: 3/20... Step: 110... Loss: 2.4485... Val Loss: 2.4827
Epoch: 4/20... Step: 120... Loss: 2.3864... Val Loss: 2.4080
Epoch: 4/20... Step: 130... Loss: 2.3184... Val Loss: 2.3621
Epoch: 4/20... Step: 140... Loss: 2.2994... Val Loss: 2.3375
Epoch: 4/20... Step: 150... Loss: 2.2488... Val Loss: 2.3042
Epoch: 5/20... Step: 160... Loss: 2.1909... Val Loss: 2.2534
Epoch: 5/20... Step: 170... Loss:

In [0]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch1.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='In', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [0]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch1.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

**Now let's see the results!**

In [0]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="Nel"))

Nel mal pion sen pel perso
  pia` ch'i l'incrima e dan pii` san don den del posti.

Le sosti sosenti selto a sonno sisini.

La` siccesti che, sol piana a la molte,
  che ' do piela den col santi, si ciute,
  che puu` cale`t che di contarti ch'asto.

Lo dior con du came si contra paco,
  che si conte li sorto arta l'irti,
  polte di porcen seser per puu` sisto.

Por si morto soltri do l'ante sucitto.

Postu sel di per che s'artri a le pan di festi,
  do par se lo saltro donor puarte,
  come che di fan den mu perdo che le pione
  centro corti sisten la sento che porch'elle,
  che 'ldo che di ciu` che dol con cono
  de l'oltin col di ciese ch'io che piasto.

Qoami se per pua` poncone sal siote;
  de santo e lo cinta ch'i le se dere,
  del mio colore e si` cantor piu che' di pasta,
  percolta a sesti an misarte cor l'isto,
  che la puar che la sio che ciesta a colso.

E lo corto liserti asciesti car pur pessa.

Lasesti di si piu` danto dicito
  e du sa che piu` le sondi santini.

Pue` de l